In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
import datasets as ds
import math
import utils as ut

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression

import pickle

### 1. Carga de información

In [2]:
critics_df = ds.get_dataset("../data/raw/rotten_tomatoes_critic_reviews.csv")

critics_df.head()

AttributeError: module 'datasets' has no attribute 'get_dataset'

* `rotten_tomatoes_link`: Sirve como identificador de la película
* `critic_name`: Nombre del critico que comento la película
* `top_critic`: Valor booleano que aclara si el crítico es un crítico destacado o no
* `publisher_name`: nombre de la editorial para la que trabaja el crítico
* `review_type`: Determina si la reseña es positiva (fresh) o negativa (rotten)
* `review_score`: Puntaje proporcionado por el crítico
* `review_date`: Fecha de la reseña
* `review_content`: Contenido de la reseña

### 2. Exploración y limpieza

**2.1. Comprensión de datos**

In [ ]:
print(f'El dataframe contiene {critics_df.shape[0]} filas y {critics_df.shape[1]} columnas.')

**2.2. Identificando nulos y duplicados**

In [ ]:
critics_df.info()

In [ ]:
print(critics_df.isnull().sum())

print("Duplicados:", critics_df.duplicated().sum())

In [ ]:
critics_df = critics_df.drop_duplicates()
critics_df = critics_df.dropna(subset=['review_content'])

* los nulis en el nombre del critico se contrastan con el de publisher name asi que no es grave y se piede mantener
* el de review score se compensa con  el de review type ya que determina si es util o no
* El de texto si es necesario arreglarlo para cuando se haga el NLP

**2.3. Eliminando información irrelevante**

In [ ]:
cr_cleaned = critics_df.copy().drop(columns=['review_date', 'review_score', 'critic_name'])

cr_cleaned.head()

* La fecha no es relevante para el estudio
* El puntaje tiene sus nulos, además de que ya hay un boolean que dice si la reseña sera positiva o negativa
* El nombre del critico no hace falta si ya esta el de la editorial que lo publica

### 3. Análisis univariante
**3.1. Dividir el Dataset en categoricos y numericos**

In [ ]:
c_df_copy = cr_cleaned.copy()
c_df_copy.dtypes.unique()

* Todas son categoricas

**3.2. Análisis sober variables categóricas**

In [ ]:
print(c_df_copy['rotten_tomatoes_link'].value_counts().head(10))
print(f"Total de productos únicos: {c_df_copy['rotten_tomatoes_link'].nunique()}")
print("------------------------------------------------------")
print(c_df_copy['publisher_name'].value_counts().head(10))
print(f"Total de editoriales únicos: {c_df_copy['publisher_name'].nunique()}")

In [ ]:
categorical_cols = [ 'top_critic', 'review_type']


for col in categorical_cols:
    c_df_copy[col].value_counts().plot(kind='bar', figsize=(6,4), title=col)
    plt.show()


* La mayoría de críticos no son destacados en una proporción casi dos veces mas que los que si lo son
* Las reseñas positivas son casi el doble a las de negativas

In [ ]:
top_n_peliculas = 10
top_n_editorial = 10

fig, axis = plt.subplots(1, 2, figsize=(16, 6))

# PRODUCTOS
top_products = c_df_copy['rotten_tomatoes_link'].value_counts().head(top_n_peliculas).index
sns.countplot(ax=axis[0], data=c_df_copy, x='rotten_tomatoes_link', order=top_products)
axis[0].set_title("Top 10 Peliculas reseñadas")
axis[0].tick_params(axis='x', rotation=90)
print("Top 10 peliculas graficados:", len(top_products))

# USUARIOS
top_users = c_df_copy['publisher_name'].value_counts().head(top_n_editorial).index
sns.countplot(ax=axis[1], data=c_df_copy, x='publisher_name', order=top_users)
axis[1].set_title("Top 10 Editoriales")
axis[1].tick_params(axis='x', rotation=45)
print("Top 10 editoriales graficados:", len(top_users))

plt.tight_layout()
plt.show()

* El top de peliculas reseñadas esta rodeando las 500 reseñas
* La editorial con mas reseñas de películas es New York Times con casi 12000, avanzando de manera descendente hasta Chicado Sun-Times con casi 8000

### 4. Análisis multivariante
**4.3. Categorico-Categorico Análisis**

In [ ]:
def cat_cat(df, cat1, cat2, top_n_cat1=None, top_n_cat2=None):
    
    if top_n_cat1:
        top_values1 = df[cat1].value_counts().nlargest(top_n_cat1).index
        df = df[df[cat1].isin(top_values1)]
    if top_n_cat2:
        top_values2 = df[cat2].value_counts().nlargest(top_n_cat2).index
        df = df[df[cat2].isin(top_values2)]
    
    ct = pd.crosstab(df[cat1], df[cat2])
    
    ct_prop = ct.div(ct.sum(axis=1).replace(0,1), axis=0)
    
    ct_prop.plot(kind='bar', stacked=True, figsize=(12,6))
    plt.xlabel(cat1)
    plt.ylabel('Proporción')
    plt.xticks(rotation=45, ha='right')
    plt.legend(title=cat2, bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()


In [ ]:
cat_cat(c_df_copy, 'rotten_tomatoes_link', 'publisher_name', top_n_cat1=top_n_peliculas, top_n_cat2=top_n_editorial)

* Parece que eFilmCritic.com es quien mas reseñas hace a las peliculas con mas reseñas del top

In [ ]:
cat_cat(c_df_copy, 'top_critic', 'publisher_name', top_n_cat2=20)

* Se ve que la mayoría de las editoriales con mas reseñas tienen criticos cuya opinion es validada, es imporatnte teniendo en cuenta que hay mas con la categoría false que con la de true

In [ ]:
cat_cat(c_df_copy, 'review_type', 'rotten_tomatoes_link', top_n_cat2=20)

* Las buenas reseñas estan distribuidas mas equitativamente entre las peliculas mientras que las malas se acumulan, esto demestra que se le da importancia a que hay pelis con mas reseñas malas que buenas

### 5. Ingeniería de características

**5.1. Preparar los datos**

* Como primera etapa vamos a mapear las reseñas de frescas o podridas como boolean

In [ ]:
c_df_copy['review_label'] = c_df_copy['review_type'].map({'Fresh': 1, 'Rotten': 0})

print(c_df_copy[['review_type', 'review_label']].head())
c_df_copy.drop(columns=['review_type'], inplace=True)


In [ ]:
c_df_copy.head()

In [ ]:
c_df_copy['top_critic_num'] = c_df_copy['top_critic'].astype(int)
c_df_copy.drop(columns=['top_critic'], inplace=True)

In [ ]:
c_df_copy['review_len'] = c_df_copy['review_content'].str.len()

# cantidad de palabras
c_df_copy['review_word_count'] = c_df_copy['review_content'].str.split().apply(len)

In [ ]:
c_df_copy.head()

**5.2. Análisis de outliers**

* Para la longitud de reseñas se tendra en cuenta que tenga un minimo de longitud para que eralmente cuente

In [ ]:
c_df_copy = c_df_copy[(c_df_copy['review_len'] >= 10) & (c_df_copy['review_len'] <= 5000)]


In [ ]:
num_cols = ['review_len', 'review_word_count', 'top_critic_num', 'review_label']

for col in num_cols:
    plt.figure(figsize=(8,2))
    sns.boxplot(x=c_df_copy[col])
    plt.title(f'Boxplot de {col}')
    plt.show()



**5.2. Dividir el train/test de marcos de datos**

In [ ]:
X = c_df_copy['review_content']  
y = c_df_copy['review_label']    

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

### 6. Transformar el texto en matriz

In [ ]:
vec_model = CountVectorizer(stop_words = "english", max_features=5000)
X_train_vec = vec_model.fit_transform(X_train)
X_test_vec = vec_model.transform(X_test)

### 7. Contruir el naive bayes

In [ ]:
nb_model = MultinomialNB()
nb_model.fit(X_train_vec, y_train)

y_pred_nb = nb_model.predict(X_test_vec)
print("Naive Bayes Accuracy:", accuracy_score(y_test, y_pred_nb))

### 8. Optimización

In [ ]:
def evaluar_modelo(model, X_train_vec, X_test_vec, y_train, y_test, nombre="Modelo"):
    model.fit(X_train_vec, y_train)
    y_pred = model.predict(X_test_vec)
    acc = accuracy_score(y_test, y_pred)
    print(f"{nombre} Accuracy: {acc:.5f}")
    return model, y_pred

In [ ]:
pipeline = Pipeline([
    ('vectorizer', CountVectorizer(stop_words='english')),
    ('nb', MultinomialNB())
])

hyperparams = {
    'vectorizer__max_features': [1000, 2000, 3000, None],
    'vectorizer__ngram_range': [(1,1), (1,2)],
    'nb__alpha': np.linspace(0.1, 2.0, 20)
}

In [ ]:
# Naive Bayes simple
vec_model = CountVectorizer(stop_words="english")
X_train_vec = vec_model.fit_transform(X_train)
X_test_vec  = vec_model.transform(X_test)

nb_model = MultinomialNB()
nb_model, y_pred_nb = evaluar_modelo(nb_model, X_train_vec, X_test_vec, y_train, y_test, "Naive Bayes")

In [ ]:
# 5. Naive Bayes optimizado con Pipeline + RandomizedSearchCV
pipeline = Pipeline([
    ('vectorizer', CountVectorizer(stop_words='english')),
    ('nb', MultinomialNB())
])

hyperparams = {
    'vectorizer__max_features': [1000, 2000, 3000, None],
    'vectorizer__ngram_range': [(1,1), (1,2)],
    'nb__alpha': np.linspace(0.1, 2.0, 20)
}

grid = RandomizedSearchCV(pipeline, hyperparams, scoring="accuracy", n_iter=20, random_state=42)
grid.fit(X_train, y_train)

best_nb_model = grid.best_estimator_
y_pred_best_nb = best_nb_model.predict(X_test)

print("Naive Bayes Optimizado Accuracy:", accuracy_score(y_test, y_pred_best_nb))
print("Mejores parámetros:", grid.best_params_)

In [ ]:
#Logistic Regression
log_model = LogisticRegression(max_iter=1000)
log_model, y_pred_log = evaluar_modelo(log_model, X_train_vec, X_test_vec, y_train, y_test, "Logistic Regression")

In [ ]:
acc_nb = accuracy_score(y_test, y_pred_best_nb)
acc_log = accuracy_score(y_test, y_pred_log)

if acc_log > acc_nb:
    modelo_final = log_model
    nombre_final = "Logistic Regression"
else:
    modelo_final = best_nb_model
    nombre_final = "Naive Bayes Optimizado"

print(f"Mejor modelo: {nombre_final} ({max(acc_nb, acc_log):.5f})")

### 9. Guardar Modelo

In [ ]:
pipeline = Pipeline([
    ("vectorizer", CountVectorizer(stop_words="english")),
    ("classifier", modelo_final)   
])

pipeline.fit(X_train, y_train)

# Guardar el pipeline completo
with open("../models/rotten_pipeline.pkl", "wb") as f:
    pickle.dump(pipeline, f)

print("Pipeline con modelo final guardado correctamente.")


In [ ]:
ruta_pipeline = "../models/rotten_pipeline.pkl"
with open(ruta_pipeline, "rb") as f:
    pipeline_cargado = pickle.load(f)

pruebas = [
    "The movie was amazing, I loved every minute of it!",
    "Absolutely terrible. I wasted two hours of my life.",
    "It was okay, some parts were good but others were boring.",
    "A masterpiece of cinematography and storytelling.",
    "Horrible acting and a very weak plot.",
]


predicciones = pipeline_cargado.predict(pruebas)

for review, pred in zip(pruebas, predicciones):
    etiqueta = "Positive" if pred == 1 else "Negative"
    print(f"Review: {review}\nPredicted sentiment: {etiqueta}\n")

